In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import AzureChatOpenAI
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langserve import add_routes
import uvicorn
import os
from dotenv import load_dotenv
from fastapi import Request
from openai import AzureOpenAI
from datetime import datetime

load_dotenv()

In [ ]:

# PDF loader
attention_loader = PyPDFLoader('../rag/attention.pdf')
attention_docs = attention_loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
attention_documents = text_splitter.split_documents(attention_docs)

# Vector embedding and vector store
embedding = OllamaEmbeddings(model="nomic-embed-text")

In [ ]:
## FAISS Vector Database

try:
    combo_db = FAISS.load_local("../storage/attention_index", embedding, allow_dangerous_deserialization=True)
except Exception as e:
    combo_db = FAISS.from_documents(attention_documents, embedding)    
    combo_db.save_local("../storage/attention_index")

db = combo_db

In [ ]:
endpoint = os.getenv("NATL_AZURE_OPENAI_ENDPOINT")
model_name = os.getenv("NATL_AZURE_OPENAI_MODEL_NAME")
deployment = os.getenv("NATL_AZURE_OPENAI_MODEL__DEPLOYMENT_NAME")

subscription_key = os.getenv("NATL_AZURE_OPENAI_KEY")
api_version = "2024-12-01-preview"

llm = AzureChatOpenAI(
    azure_deployment=deployment,
    openai_api_version=api_version,
    azure_endpoint=endpoint,
    api_key=subscription_key,
    temperature=0.7
)

In [ ]:
## Design chat prompt

prompt = ChatPromptTemplate.from_template("""
                                          Answer the following question based only on the provided context.
                                          Think step by step before providing a detailed answer.
                                          <context>
                                          {context}
                                          </context>
                                          Question: {input}""")

In [ ]:
## Chain introduction
## Create stuff documents chain

document_chain = create_stuff_documents_chain(llm=llm, prompt=prompt)

retriever = db.as_retriever()

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [ ]:
user_input = "what is an encoder?"

try:
    response = retrieval_chain.invoke({"input": user_input})
except Exception as e:
    print("error: ", str(e))

In [ ]:
response['answer']